In [1]:
# Task for the 15th of December, 2017



# These two lines are necessary only if gempy is not installed
import sys, os
sys.path.append("../..")

import gempy as gp
%matplotlib inline

import numpy as np
import pandas as pn
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# Imports of libraries needed in the following code

from mpl_toolkits.mplot3d import Axes3D
import itertools
from scipy.spatial import ConvexHull

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
# However as we increase the resolution the model looks better and better! This I guess it is the marching cube error
new_grid = gp.GridClass()
res = 40

In [5]:
new_grid.create_custom_grid(np.array([[2,3,4]]))

array([[2, 3, 4]])

In [7]:
# Importing the data from csv files and settign extent and resolution
geo_data = gp.create_data([0,2000,0,2000,-2000,0],[ 60,60,60],
                         path_f = os.pardir+"/input_data/FabLessPoints_Foliations.csv",
                         path_i = os.pardir+"/input_data/FabLessPoints_Points.csv")

../../gempy/data_management.py:61: UserWarning: path_f is deprecated use instead path_o
  warnings.warn('path_f is deprecated use instead path_o')


In [8]:
# Assigning series to formations as well as their order (timewise)
gp.set_series(geo_data, {"fault":'MainFault', 
                      "Rest": ('SecondaryReservoir','Seal', 'Reservoir', 'Overlying')},
                       order_series = ["fault", 'Rest'],
                       order_formations=['MainFault', 
                                         'SecondaryReservoir', 'Seal','Reservoir', 'Overlying',
                                         ]) 


In [9]:
import theano.tensor as T
import theano
interp_data = gp.InterpolatorData(geo_data, u_grade=[1],
                                  output='geology', dtype='float64',
                                   verbose=['scalar_field_iter', 'block_series'],
                                  compile_theano=True)

Compiling theano function...
Compilation Done!
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  1


In [12]:
(new_grid.values  - interp_data.centers.as_matrix()) / interp_data.rescaling_factor + 0.5001

array([[0.20574102564102564, 0.24445897435897435, 0.7575358974358974]], dtype=object)

In [13]:
x_to_interpolate = np.vstack((depth_grid_res,
                              interp_data.interpolator.pandas_rest_layer_points[['X', 'Y', 'Z']].as_matrix(),
                              interp_data.interpolator.pandas_ref_layer_points_rep[['X', 'Y', 'Z']].as_matrix()))

NameError: name 'depth_grid_res' is not defined

In [ ]:
# Creating the drift matrix.
universal_matrix = np.vstack((x_to_interpolate.T,
                             (x_to_interpolate ** 2).T,
                              x_to_interpolate[:, 0] * x_to_interpolate[:, 1],
                              x_to_interpolate[:, 0] * x_to_interpolate[:, 2],
                              x_to_interpolate[:, 1] * x_to_interpolate[:, 2],
                              ))

interp_data.interpolator.tg.grid_val_T.set_value(np.cast['float32'](x_to_interpolate + 10e-9))

            # Universal grid
interp_data.interpolator.tg.universal_grid_matrix_T.set_value(np.cast['float32'](universal_matrix + 1e-10))

In [14]:
interp_data.interpolator.dtype

'float64'

In [ ]:


# calculate centroids of voxels in domain 
centroids, dividex, dividey, dividez = computeCentroids(xdown, xup, ydown, yup, zdown, zup, res)

# Create a new grid object with the new resolution
new_grid.create_regular_grid_3d([xdown,xup,ydown,yup,zdown,zup],[res, res, res])

# Setting the new grid to the geodata
gp.set_grid(geo_data, new_grid)

# Update the interp_data
interp_data.update_interpolator(geo_data)

# Recomputiong
lith, fault = gp.compute_model(interp_data)

ver_s, sim_s = gp.get_surfaces(interp_data,lith[1], None, original_scale=True)
